In [28]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import tsfel
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import os

In [6]:
X_train_featurised = pd.read_csv("./UCI HAR Dataset/train/X_train.txt", sep='\s+',header=None)
y_train_featurised = pd.read_csv("./UCI HAR Dataset/train/y_train.txt", sep='\s+',header=None)

print(X_train_featurised.shape)

(7352, 561)


In [11]:
time = 10
offset = 100
folders = ["LAYING","SITTING","STANDING","WALKING","WALKING_DOWNSTAIRS","WALKING_UPSTAIRS"]
classes = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}

combined_dir = os.path.join("Real Data")

X_test=[]
y_test=[]
dataset_dir = os.path.join(combined_dir,"Test")

for folder in folders:
    files = os.listdir(os.path.join(dataset_dir,folder))
    for file in files:

        df = pd.read_csv(os.path.join(dataset_dir,folder,file),sep=",",header=0)
        df = df[offset:offset+time*50]
        X_test.append(df.values)
        y_test.append(classes[folder])

X_real_test = np.array(X_test)
y_real_test = np.array(y_test)

In [12]:
cfg = tsfel.get_features_by_domain() # retrieves all features
X_real_test_featurised = tsfel.time_series_features_extractor(cfg, X_real_test, fs=50)

In [23]:
print(X_real_test_featurised.shape)
print(X_train_featurised.shape)

(24, 825)
(7352, 561)


In [27]:
model = DecisionTreeClassifier()
model.fit(X_train_featurised, y_train_featurised)
y_pred = model.predict(X_real_test_featurised.iloc[0:24,0:561])
accuracy = accuracy_score(y_real_test, y_pred)

precision = precision_score(y_real_test, y_pred, average='weighted')

recall = recall_score(y_real_test, y_pred, average='weighted')

conf_matrix = confusion_matrix(y_real_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

print("\nClassification Report:")
print(classification_report(y_real_test, y_pred))

Accuracy: 0.1667
Precision: 0.0278
Recall: 0.1667

Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         4
           3       0.17      1.00      0.29         4
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         4
           6       0.00      0.00      0.00         4

    accuracy                           0.17        24
   macro avg       0.03      0.17      0.05        24
weighted avg       0.03      0.17      0.05        24



C:\Users\ganes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
C:\Users\ganes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ganes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parame

In [30]:
seed = 4
X_train,X_test,y_train,y_test = train_test_split(X_real_test,y_real_test,test_size=0.3,random_state=seed,stratify=y_real_test)

print("Training data shape: ",X_train.shape)
print("Testing data shape: ",X_test.shape)

Training data shape:  (16, 500, 5)
Testing data shape:  (8, 500, 5)


In [31]:
cfg = tsfel.get_features_by_domain() # retrieves all features
X_real_train_featurised = tsfel.time_series_features_extractor(cfg, X_train, fs=50)
X_real_test_featurised = tsfel.time_series_features_extractor(cfg, X_test, fs=50)

In [32]:
model = DecisionTreeClassifier()
model.fit(X_real_train_featurised, y_train)
y_pred = model.predict(X_real_test_featurised)
accuracy = accuracy_score(y_test, y_pred)

precision = precision_score(y_test, y_pred, average='weighted')

recall = recall_score(y_test, y_pred, average='weighted')

conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.5000
Precision: 0.4688
Recall: 0.5000

Classification Report:
              precision    recall  f1-score   support

           1       0.25      1.00      0.40         1
           2       1.00      0.50      0.67         2
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         1
           5       0.50      1.00      0.67         1
           6       1.00      1.00      1.00         1

    accuracy                           0.50         8
   macro avg       0.46      0.58      0.46         8
weighted avg       0.47      0.50      0.42         8



C:\Users\ganes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ganes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ganes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati